In [1]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
import pandas as pd
from glob import glob

Using TensorFlow backend.


In [2]:
#function to load the dataset
def load_dataset(path):
    data = load_files(path)
    fish_files = np.array(data['filenames'])
    fish_target = np_utils.to_categorical(np.array(data['target']), 8)
    return fish_files,fish_target

In [3]:
#let's load the training-data
train_files, train_targets = load_dataset('data/train')

#let's load the teting-data
test_files, _ = load_dataset('data/test')

#print the number of samples in test and trainin sets
print ("There are %d images in training dataset"%len(train_files))
print ("There are %d images in the training set"%len(test_files))

There are 3777 images in training dataset
There are 13153 images in the training set


In [5]:
from keras.preprocessing import image
from tqdm import tqdm

#converting image to tensor
def path_to_tensor(img_path):
    # loads RGB image
    img = image.load_img(img_path, target_size=(224,224))
    #convering the image to 3-D tensor with shape (224,224,3)
    x = image.img_to_array(img)
    #convert 3D tensor to 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [6]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True 

#preprocessing the data
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|███████████████████████████████████████████████████████████████████████████| 13153/13153 [02:06<00:00, 103.80it/s]


In [7]:
train_tensors = paths_to_tensor(train_files).astype('float32')/255

100%|██████████████████████████████████████████████████████████████████████████████| 3777/3777 [01:38<00:00, 41.52it/s]


In [8]:
#shape of the tensor
print(np.shape(train_tensors))

(3777, 224, 224, 3)


# Let's have the Benchmark Model

In [9]:
# Let's have a Benchmark model

from keras.layers import Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout
from keras.models import Sequential

benchmark = Sequential()

# model Convolution layer
benchmark.add(Conv2D(filters=16,kernel_size=2,strides=1,activation='relu',input_shape=(224,224,3)))
# Max Pooling layer to reduce the dimensionality
benchmark.add(MaxPooling2D(pool_size=2,strides=2))
#Dropout layer, for turning off each node with the probability of 0.3
benchmark.add(Dropout(0.3))
benchmark.add(Conv2D(filters=32, kernel_size=2,strides=1,activation='relu'))
benchmark.add(Dropout(0.3))
benchmark.add(GlobalAveragePooling2D())
#A fully connected dense layer with 8 nodes (no of classes of fish)
benchmark.add(Dense(8,activation='softmax'))
benchmark.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 223, 223, 16)      208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 16)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 110, 110, 32)      2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 110, 110, 32)      0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 264       
Total para

In [10]:
benchmark.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
from keras.callbacks import ModelCheckpoint, EarlyStopping


epochs = 10

#checkpointer saves the best weights.
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.benchmark.hdf5', verbose=1, save_best_only=True)

benchmark.fit(train_tensors, train_targets, batch_size=20, epochs=epochs, callbacks=[checkpointer], validation_split=0.2, verbose=1)

Train on 3021 samples, validate on 756 samples
Epoch 1/10
3021/3021 [==============================] - ETA: 16:33 - loss: 1.9862 - acc: 0.05 - ETA: 5:30 - loss: 1.9573 - acc: 0.1833 - ETA: 3:17 - loss: 1.9341 - acc: 0.300 - ETA: 2:20 - loss: 1.9107 - acc: 0.350 - ETA: 1:48 - loss: 1.9088 - acc: 0.361 - ETA: 1:28 - loss: 1.8724 - acc: 0.400 - ETA: 1:14 - loss: 1.8683 - acc: 0.403 - ETA: 1:04 - loss: 1.8572 - acc: 0.406 - ETA: 56s - loss: 1.8547 - acc: 0.400 - ETA: 50s - loss: 1.8548 - acc: 0.39 - ETA: 45s - loss: 1.8286 - acc: 0.40 - ETA: 41s - loss: 1.8137 - acc: 0.41 - ETA: 37s - loss: 1.7956 - acc: 0.41 - ETA: 34s - loss: 1.8016 - acc: 0.41 - ETA: 31s - loss: 1.7868 - acc: 0.42 - ETA: 29s - loss: 1.7766 - acc: 0.42 - ETA: 27s - loss: 1.7811 - acc: 0.41 - ETA: 25s - loss: 1.7723 - acc: 0.42 - ETA: 24s - loss: 1.7620 - acc: 0.42 - ETA: 22s - loss: 1.7607 - acc: 0.42 - ETA: 21s - loss: 1.7520 - acc: 0.42 - ETA: 20s - loss: 1.7526 - acc: 0.41 - ETA: 19s - loss: 1.7438 - acc: 0.42 - ETA: 

3021/3021 [==============================] - ETA: 4s - loss: 1.5784 - acc: 0.450 - ETA: 4s - loss: 1.5690 - acc: 0.450 - ETA: 4s - loss: 1.5960 - acc: 0.400 - ETA: 4s - loss: 1.5987 - acc: 0.421 - ETA: 4s - loss: 1.6391 - acc: 0.400 - ETA: 4s - loss: 1.6314 - acc: 0.422 - ETA: 4s - loss: 1.6402 - acc: 0.426 - ETA: 4s - loss: 1.6452 - acc: 0.426 - ETA: 4s - loss: 1.6683 - acc: 0.411 - ETA: 4s - loss: 1.6367 - acc: 0.439 - ETA: 4s - loss: 1.6214 - acc: 0.447 - ETA: 3s - loss: 1.6319 - acc: 0.437 - ETA: 3s - loss: 1.6206 - acc: 0.438 - ETA: 3s - loss: 1.6144 - acc: 0.446 - ETA: 3s - loss: 1.6165 - acc: 0.446 - ETA: 3s - loss: 1.6152 - acc: 0.448 - ETA: 3s - loss: 1.6114 - acc: 0.453 - ETA: 3s - loss: 1.5921 - acc: 0.461 - ETA: 3s - loss: 1.5968 - acc: 0.459 - ETA: 3s - loss: 1.5984 - acc: 0.456 - ETA: 3s - loss: 1.6016 - acc: 0.456 - ETA: 3s - loss: 1.6050 - acc: 0.453 - ETA: 3s - loss: 1.6086 - acc: 0.448 - ETA: 3s - loss: 1.6065 - acc: 0.451 - ETA: 3s - loss: 1.6013 - acc: 0.452 - ETA: 

3021/3021 [==============================] - ETA: 5s - loss: 1.6758 - acc: 0.500 - ETA: 4s - loss: 1.4223 - acc: 0.583 - ETA: 4s - loss: 1.5220 - acc: 0.510 - ETA: 4s - loss: 1.5340 - acc: 0.471 - ETA: 4s - loss: 1.5321 - acc: 0.477 - ETA: 4s - loss: 1.5826 - acc: 0.463 - ETA: 4s - loss: 1.5686 - acc: 0.473 - ETA: 4s - loss: 1.5489 - acc: 0.480 - ETA: 4s - loss: 1.5405 - acc: 0.488 - ETA: 4s - loss: 1.5418 - acc: 0.489 - ETA: 4s - loss: 1.5387 - acc: 0.495 - ETA: 4s - loss: 1.5514 - acc: 0.484 - ETA: 4s - loss: 1.5385 - acc: 0.492 - ETA: 4s - loss: 1.5432 - acc: 0.492 - ETA: 3s - loss: 1.5365 - acc: 0.498 - ETA: 3s - loss: 1.5438 - acc: 0.493 - ETA: 3s - loss: 1.5597 - acc: 0.481 - ETA: 3s - loss: 1.5543 - acc: 0.478 - ETA: 3s - loss: 1.5494 - acc: 0.483 - ETA: 3s - loss: 1.5549 - acc: 0.480 - ETA: 3s - loss: 1.5545 - acc: 0.478 - ETA: 3s - loss: 1.5544 - acc: 0.477 - ETA: 3s - loss: 1.5643 - acc: 0.472 - ETA: 3s - loss: 1.5595 - acc: 0.474 - ETA: 3s - loss: 1.5540 - acc: 0.475 - ETA: 

3021/3021 [==============================] - ETA: 4s - loss: 1.5152 - acc: 0.450 - ETA: 4s - loss: 1.5455 - acc: 0.483 - ETA: 4s - loss: 1.6316 - acc: 0.460 - ETA: 4s - loss: 1.6182 - acc: 0.471 - ETA: 4s - loss: 1.5828 - acc: 0.483 - ETA: 4s - loss: 1.5936 - acc: 0.463 - ETA: 4s - loss: 1.6085 - acc: 0.450 - ETA: 4s - loss: 1.6052 - acc: 0.436 - ETA: 4s - loss: 1.6145 - acc: 0.432 - ETA: 4s - loss: 1.5992 - acc: 0.426 - ETA: 4s - loss: 1.5911 - acc: 0.427 - ETA: 4s - loss: 1.5775 - acc: 0.438 - ETA: 4s - loss: 1.5714 - acc: 0.445 - ETA: 4s - loss: 1.5664 - acc: 0.450 - ETA: 3s - loss: 1.5537 - acc: 0.448 - ETA: 3s - loss: 1.5353 - acc: 0.460 - ETA: 3s - loss: 1.5431 - acc: 0.456 - ETA: 3s - loss: 1.5357 - acc: 0.464 - ETA: 3s - loss: 1.5439 - acc: 0.462 - ETA: 3s - loss: 1.5428 - acc: 0.459 - ETA: 3s - loss: 1.5467 - acc: 0.460 - ETA: 3s - loss: 1.5393 - acc: 0.463 - ETA: 3s - loss: 1.5452 - acc: 0.460 - ETA: 3s - loss: 1.5405 - acc: 0.463 - ETA: 3s - loss: 1.5399 - acc: 0.464 - ETA: 

3021/3021 [==============================] - ETA: 4s - loss: 1.2671 - acc: 0.650 - ETA: 4s - loss: 1.3895 - acc: 0.533 - ETA: 4s - loss: 1.4639 - acc: 0.490 - ETA: 4s - loss: 1.4637 - acc: 0.492 - ETA: 4s - loss: 1.5018 - acc: 0.472 - ETA: 4s - loss: 1.4581 - acc: 0.500 - ETA: 4s - loss: 1.4396 - acc: 0.507 - ETA: 4s - loss: 1.4870 - acc: 0.486 - ETA: 4s - loss: 1.4906 - acc: 0.488 - ETA: 4s - loss: 1.4989 - acc: 0.478 - ETA: 4s - loss: 1.4911 - acc: 0.483 - ETA: 4s - loss: 1.4905 - acc: 0.476 - ETA: 4s - loss: 1.4935 - acc: 0.480 - ETA: 3s - loss: 1.4948 - acc: 0.475 - ETA: 3s - loss: 1.4853 - acc: 0.479 - ETA: 3s - loss: 1.4750 - acc: 0.480 - ETA: 3s - loss: 1.4655 - acc: 0.486 - ETA: 3s - loss: 1.4878 - acc: 0.478 - ETA: 3s - loss: 1.4834 - acc: 0.477 - ETA: 3s - loss: 1.4929 - acc: 0.478 - ETA: 3s - loss: 1.4955 - acc: 0.478 - ETA: 3s - loss: 1.4935 - acc: 0.479 - ETA: 3s - loss: 1.4960 - acc: 0.480 - ETA: 3s - loss: 1.4950 - acc: 0.481 - ETA: 3s - loss: 1.4906 - acc: 0.482 - ETA: 

## Predictions for Benchmark

### Loading the  weights of Benchmark model

In [12]:
benchmark.load_weights('saved_models/weights.best.benchmark.hdf5')

### Predictions

In [13]:
benchmark_model_prediction = [benchmark.predict(np.expand_dims(img_tensor, axis=0)) for img_tensor in test_tensors]

### Processing the Predictions

In [14]:
#visaulizing the array
print(benchmark_model_prediction[:][0])

#swapping the axes of the benchmark_model_prediction for easy handling
benchmark_model_prediction = np.swapaxes(benchmark_model_prediction,0,1)

#creating a pandas dataframe for with benchmark model's prediction
df_pred_model1 = pd.DataFrame(benchmark_model_prediction[0][:], columns=['ALB','BET','DOL','LAG','NoF','OTHER','SHARK','YFT'])

#first five rows of df_pred_model1 dataframe
print(df_pred_model1[:5])

[[0.42553875 0.05191546 0.03911333 0.01771741 0.06787238 0.06665134
  0.0635898  0.26760146]]
        ALB       BET       DOL       LAG       NoF     OTHER     SHARK  \
0  0.425539  0.051915  0.039113  0.017717  0.067872  0.066651  0.063590   
1  0.479572  0.043038  0.030075  0.020164  0.235390  0.055788  0.014474   
2  0.432765  0.043701  0.026840  0.024427  0.299017  0.066578  0.013751   
3  0.506477  0.047196  0.027785  0.024373  0.201054  0.063204  0.020254   
4  0.464244  0.044821  0.029505  0.022817  0.252187  0.061723  0.015182   

        YFT  
0  0.267601  
1  0.121501  
2  0.092921  
3  0.109657  
4  0.109522  


In [15]:
#test_files[0]

#extracting relevant name of the image from the full-path of image
image_names = [test_files[i][15:] for i in range(len(test_files))]

In [16]:
#adjusting the filename of the image to match the submission guidelines
for i in range(13153):
    if image_names[i][5]=='_':
        image_names[i] = "test_stg2/" + image_names[i]

In [17]:
image_names[1323]

'test_stg2/image_04056.jpg'

In [18]:
#adding image names to our dataframe
df_pred_model1['image'] = pd.DataFrame(image_names)

#reindexing the dataframe
df_pred_model1 = df_pred_model1.reindex_axis(['image','ALB','BET','DOL','LAG','NoF','OTHER','SHARK','YFT'], axis=1)

#printing the first five rows of dataframe
print(df_pred_model1[:5])

C:\Users\user\AppData\Local\conda\conda\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """


                       image       ALB       BET       DOL       LAG  \
0  test_stg2/image_10973.jpg  0.425539  0.051915  0.039113  0.017717   
1  test_stg2/image_00175.jpg  0.479572  0.043038  0.030075  0.020164   
2  test_stg2/image_09645.jpg  0.432765  0.043701  0.026840  0.024427   
3              img_02920.jpg  0.506477  0.047196  0.027785  0.024373   
4  test_stg2/image_09349.jpg  0.464244  0.044821  0.029505  0.022817   

        NoF     OTHER     SHARK       YFT  
0  0.067872  0.066651  0.063590  0.267601  
1  0.235390  0.055788  0.014474  0.121501  
2  0.299017  0.066578  0.013751  0.092921  
3  0.201054  0.063204  0.020254  0.109657  
4  0.252187  0.061723  0.015182  0.109522  


In [19]:
df_pred_model1.tail(10)

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
13143,test_stg2/image_05875.jpg,0.515315,0.041940,0.032962,0.016914,0.178082,0.047733,0.015448,0.151607
13144,test_stg2/image_04374.jpg,0.540486,0.040701,0.034733,0.013437,0.118196,0.038110,0.019496,0.194842
13145,test_stg2/image_07892.jpg,0.452034,0.052957,0.033044,0.021281,0.106285,0.081277,0.050915,0.202207
13146,test_stg2/image_09226.jpg,0.450020,0.052148,0.036669,0.020324,0.128799,0.070184,0.037010,0.204846
13147,test_stg2/image_04860.jpg,0.276634,0.042274,0.030476,0.024683,0.448714,0.074510,0.009589,0.093120
13148,img_07578.jpg,0.501905,0.041839,0.035838,0.014108,0.145416,0.042610,0.017613,0.200671
13149,test_stg2/image_03265.jpg,0.556458,0.035502,0.025563,0.013570,0.167819,0.042091,0.013229,0.145767
13150,test_stg2/image_09846.jpg,0.437534,0.036371,0.027692,0.015496,0.283582,0.052302,0.009798,0.137225
13151,test_stg2/image_10800.jpg,0.509217,0.047430,0.028931,0.018075,0.084743,0.067471,0.049382,0.194752
13152,test_stg2/image_02733.jpg,0.484187,0.037991,0.033073,0.013554,0.196655,0.043481,0.011850,0.179210


##  .csv file for submission

In [20]:
df_pred_model1.to_csv('submission1-benchmark.csv',index=False)

---